In [1]:
from netin.models import DPAHModel

In [2]:
#TODO: check icm diffusion model, error when generating the random values, always reusing the same ones over and over again!!!!
#TODO: reduce seed size, to have more variability in the results
#TODO: Change budget amount, and see if one then have more variability in the results
#TODO: Change the density of the graph

In [3]:
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set


from src.seed import (seed_degree, 
    seed_polarized, 
    seed_random, 
    seed_polarized_centrality_mixed, 
    seed_polarized_centrality, 
    seed_polarized_degree, 
    seed_centrality_mixed, 
    seed_centrality, 
    seed_influence_maximization
)


from evaluation import evaluate_all_seeds

This model connects a new node to the existing nodes biased by their group assignment. Homophily values of $h > 0.5$ indicate homophily, meaning that nodes prefer to connect to their own group. Heterophily is specified by $h < 0.5$. Nodes tend to connect to the other group.

Instead of the number of links per each new node, the directed graph models parameterize the density of the final network by the parameter $d$. Additionally, we need to define activity levels for outgoing links for both the minority and majority group as $plo_m/M$

In [4]:
N = 10_000 # Number of total nodes
f_m = 0.3 # Fraction of minority nodes
h = 0.9 # Homophilic
d = 0.002 # desity of the graph
plo = 1.9
#final number of edges will be: e = d * n (n-1)

In [5]:
# DPAH graphs:
# Homophilic h > 0.5
# Neutral h = 0.5
# Heterophilic h < 0.5

homophily_values = [0.1, 0.5, 0.9]

In [6]:
dpa_model = DPAHModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h, seed=12345)
dpa_graph = dpa_model.simulate()

In [7]:
G = dpa_graph.to_nxgraph() # This creates a copy of the graph

for node, attr in G.nodes(data=True):
    attr['color'] = attr.pop('minority')

for node in G.nodes():
        in_degree = G.in_degree(node)
        for neighbor in G.successors(node):
            G[node][neighbor]["weight"] = 1 / (2 * in_degree) if in_degree > 0 else 1

In [8]:
seed_size = G.number_of_nodes() // 20
print(f"Seed size: {seed_size}")
k = seed_size
max_iter = 1000
budget = 3 * seed_size

Seed size: 500


In [9]:
# Define your seed functions
seed_functions = {
    "Random": seed_random,
    "MIA": lambda G, seed_size: seed_influence_maximization(G, seed_size, 500, 8, 0),
    "Degree": seed_degree,
    "Centrality": seed_centrality,
    "Centrality Mixed": seed_centrality_mixed,
    "Polarized": lambda G, seed_size: seed_polarized(G, seed_size, 0),
    "Polarized Degree": lambda G, seed_size: seed_polarized_degree(G, seed_size, 0),
    "Polarized Centrality": lambda G, seed_size: seed_polarized_centrality(G, seed_size, 0),
    "Polarized Centrality Mixed": lambda G, seed_size: seed_polarized_centrality_mixed(G, seed_size, 0),
}

# Example of how to call the function
all_results_df = evaluate_all_seeds(G, seed_functions, seed_size, k, max_iter, budget, "netin_09_3")

# Display the combined results
print("Combined Results for All Seed Functions")
all_results_df

Running evaluation for seed function: Random
    Running evaluation for graph modification: Original Graph


100%|██████████| 1000/1000 [00:30<00:00, 32.39it/s]


    Running evaluation for graph modification: Random


100%|██████████| 1000/1000 [00:40<00:00, 24.73it/s]


    Running evaluation for graph modification: Degree


...

100%|██████████| 1000/1000 [00:47<00:00, 20.87it/s]


    Running evaluation for graph modification: Custom V2


100%|██████████| 1000/1000 [00:43<00:00, 23.11it/s]

Combined Results for All Seed Functions


Metric,Seed Function,Graph Modification,Avg Activated Nodes,Activated Nodes Std Dev,"Avg Activated Nodes, Color (0)","Activated Nodes Std Dev, Color (0)","Avg Activated Nodes, Color (1)","Activated Nodes Std Dev, Color (1)",Avg Color Activation Count,Color Activation Count Std Dev,Number of Nodes,Number of Edges
0,Centrality,Original Graph,626.441,38.383,578.915,28.326,47.526,15.577,10.168,9.412,10000.0,199980.0
1,Centrality,PrefAtt,9680.000,0.000,6718.000,0.000,2962.000,0.000,3790.043,437.740,10000.0,201480.0
2,Centrality,Jaccard,9680.000,0.000,6718.000,0.000,2962.000,0.000,9706.306,2475.477,10000.0,201480.0
3,Centrality,Degree,9680.000,0.000,6718.000,0.000,2962.000,0.000,6612.405,327.575,10000.0,201480.0
4,Centrality,TopK,9680.000,0.000,6718.000,0.000,2962.000,0.000,7937.529,1476.415,10000.0,201480.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Random,TopK,9696.000,0.000,6729.000,0.000,2967.000,0.000,6204.092,282.322,10000.0,201480.0
77,Random,KKT,9696.000,0.000,6729.000,0.000,2967.000,0.000,7922.416,1412.068,10000.0,201480.0
78,Random,Random,1392.250,111.704,935.160,74.479,457.090,45.844,284.726,45.412,10000.0,201480.0
79,Random,Custom,1544.845,102.967,863.726,66.373,681.119,44.066,454.263,47.003,10000.0,201480.0


In [10]:
#all_results_df.to_csv(f"test.csv", index=False)
all_results_df.to_csv(f"results/results_netin_{N}_nodes_{h}_hvalue_{d}_density_{plo}_plo_3.csv", index=False)